<a href="https://colab.research.google.com/github/ZenithZack/lda_models/blob/main/LDATraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
data = pd.read_csv('/content/abcnews-date-text.csv', error_bad_lines=False)

In [ ]:
data_text = data[['headline_text']]
#print(type(data_text))
#print(data_text)
data_text['index'] = data_text.index
documents = data_text
#print(type(documents))

In [ ]:
print(len(documents))
print(documents[0:5])

127958
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# Stemmer is an object and stem is a function (Stemming is carried out). The stem basically takes string as argument
# Had to find the right stemmer class on basis of the number of positional arguments
def lemmatize_stemming(text):
  return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [ ]:
def preprocess(text):
  result = []
  for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
      result.append(lemmatize_stemming(token))
  return result

In [ ]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print(type(doc_sample))
print(doc_sample)

print('original document: ')
words = []
for word in doc_sample.split(' '):
  words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

<class 'str'>
ratepayers group wants compulsory local govt voting
original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [ ]:
documents = documents.dropna(subset=['headline_text'])
processed_docs = documents['headline_text'].map(preprocess)

In [ ]:
processed_docs[:4311]

0                      [decid, commun, broadcast, licenc]
1                                      [wit, awar, defam]
2                  [call, infrastructur, protect, summit]
3                             [staff, aust, strike, rise]
4                    [strike, affect, australian, travel]
                              ...                        
4306                          [queensland, cyclon, watch]
4307                                [question, rais, gmo]
4308                        [rain, hamper, goat, product]
4309                        [rain, help, dampen, bushfir]
4310    [ratepay, group, want, compulsori, local, govt...
Name: headline_text, Length: 4311, dtype: object

In [ ]:
# Here the argument 'preprocessed_docs' is a 'list of lists'
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
# print(len(dictionary.keys()))
for k, v in dictionary.iteritems():
  print(k, v)
  count += 1
  if count > 5:
    break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam


In [ ]:
dictionary.filter_extremes(no_below = 15, no_above = 0.5, keep_n = 100000)

In [ ]:
# Extra lines of code for generating N-grams
from gensim.models.phrases import Phraser, Phrases
bigram = Phrases(processed_docs, min_count = 1,threshold = 100)
trigrams = Phrases(bigram[processed_docs], min_count = 5, threshold = 100)
print(bigram[processed_docs[0]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['decid', 'commun', 'broadcast', 'licenc']


In [ ]:
# Continued implementation of above (to save on RAM)
print(trigrams[processed_docs[5]])

['ambiti', 'olsson', 'win', 'tripl', 'jump']


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4309]
#bow_doc_4310 = bow_corpus[4310]
#for i in range(len(bow_doc_4310)):
    #print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               #dictionary[bow_doc_4310[i][0]], 
#bow_doc_4310[i][1]))

[(68, 1), (102, 1), (443, 1), (2997, 1)]

In [ ]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
#print(tfidf)
corpus_tfidf = tfidf[bow_corpus]
print(corpus_tfidf)
from pprint import pprint

for doc in corpus_tfidf:
  pprint(doc)
  break

[(0, 0.5871107924946463),
 (1, 0.39059932887674903),
 (2, 0.4897672242374202),
 (3, 0.5126998612054761)]


In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)
  File "/usr/local/lib/python3.7/dist-packages/gensim/models/ldamulticore.py", line 333, in worker_e_step
    worker_lda.do_estep(chunk)  # TODO: auto-tune alpha?
  File "/usr/local/lib/python3.7/dist-packages/gensim/models/ldamulticore.py", line 333, 

KeyboardInterrupt: ignored

In [ ]:
for idx, topic in lda_model.print_topics(-1):
  print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.061*"polic" + 0.029*"kill" + 0.024*"crash" + 0.018*"death" + 0.016*"investig" + 0.016*"miss" + 0.016*"attack" + 0.015*"probe" + 0.013*"bomb" + 0.012*"search"
Topic: 1 
Words: 0.020*"rise" + 0.019*"price" + 0.018*"say" + 0.017*"govt" + 0.017*"health" + 0.014*"farmer" + 0.012*"fund" + 0.011*"hospit" + 0.011*"fuel" + 0.010*"flood"
Topic: 2 
Words: 0.027*"plan" + 0.017*"nation" + 0.016*"urg" + 0.016*"hous" + 0.015*"public" + 0.015*"elect" + 0.015*"power" + 0.014*"continu" + 0.014*"govt" + 0.013*"council"
Topic: 3 
Words: 0.018*"concern" + 0.017*"govt" + 0.016*"protest" + 0.015*"school" + 0.014*"labor" + 0.013*"union" + 0.012*"defend" + 0.010*"inquiri" + 0.010*"bird" + 0.010*"plan"
Topic: 4 
Words: 0.023*"fear" + 0.017*"push" + 0.015*"break" + 0.014*"worker" + 0.014*"famili" + 0.013*"fish" + 0.012*"telstra" + 0.011*"sale" + 0.010*"feder" + 0.010*"govt"
Topic: 5 
Words: 0.015*"coast" + 0.015*"year" + 0.014*"gold" + 0.014*"market" + 0.013*"open" + 0.013*"record" + 0.010*"wo

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
  print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.011*"east" + 0.010*"benefit" + 0.010*"violenc" + 0.008*"stab" + 0.007*"evacu" + 0.006*"wood" + 0.006*"eagl" + 0.006*"leak" + 0.006*"timor" + 0.005*"dog"
Topic: 1 
Words: 0.015*"terror" + 0.011*"plane" + 0.010*"whale" + 0.008*"nurs" + 0.008*"corbi" + 0.008*"number" + 0.007*"address" + 0.007*"crash" + 0.007*"anti" + 0.007*"light"
Topic: 2 
Words: 0.016*"bird" + 0.011*"doubt" + 0.007*"alcohol" + 0.007*"pacif" + 0.007*"medic" + 0.006*"mix" + 0.006*"south" + 0.006*"africa" + 0.006*"expans" + 0.006*"chelsea"
Topic: 3 
Words: 0.017*"toll" + 0.015*"kill" + 0.010*"death" + 0.009*"bomb" + 0.009*"road" + 0.009*"suicid" + 0.008*"escap" + 0.008*"blast" + 0.007*"accid" + 0.007*"hurt"
Topic: 4 
Words: 0.013*"drive" + 0.012*"drink" + 0.008*"retir" + 0.008*"hostag" + 0.007*"coal" + 0.007*"driver" + 0.006*"energi" + 0.006*"patient" + 0.006*"young" + 0.006*"polic"
Topic: 5 
Words: 0.018*"price" + 0.011*"fuel" + 0.010*"market" + 0.008*"stock" + 0.008*"lion" + 0.007*"petrol" + 0.007*"hig

In [ ]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [ ]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
  print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5599225759506226	 
Topic: 0.027*"plan" + 0.017*"nation" + 0.016*"urg" + 0.016*"hous" + 0.015*"public" + 0.015*"elect" + 0.015*"power" + 0.014*"continu" + 0.014*"govt" + 0.013*"council"

Score: 0.20732003450393677	 
Topic: 0.023*"fear" + 0.017*"push" + 0.015*"break" + 0.014*"worker" + 0.014*"famili" + 0.013*"fish" + 0.012*"telstra" + 0.011*"sale" + 0.010*"feder" + 0.010*"govt"

Score: 0.14525026082992554	 
Topic: 0.036*"charg" + 0.034*"court" + 0.032*"face" + 0.021*"accus" + 0.020*"jail" + 0.017*"murder" + 0.016*"drug" + 0.016*"case" + 0.013*"trial" + 0.013*"terror"

Score: 0.012503069825470448	 
Topic: 0.034*"council" + 0.024*"water" + 0.021*"boost" + 0.017*"plan" + 0.014*"offer" + 0.012*"fund" + 0.012*"chang" + 0.012*"govt" + 0.010*"delay" + 0.010*"welcom"

Score: 0.012502149678766727	 
Topic: 0.018*"concern" + 0.017*"govt" + 0.016*"protest" + 0.015*"school" + 0.014*"labor" + 0.013*"union" + 0.012*"defend" + 0.010*"inquiri" + 0.010*"bird" + 0.010*"plan"

Score: 0.01250147633

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5791507363319397	 
Topic: 0.011*"govt" + 0.011*"fund" + 0.011*"health" + 0.009*"urg" + 0.008*"plan" + 0.008*"servic" + 0.008*"council" + 0.007*"boost" + 0.006*"indigen" + 0.006*"seek"

Score: 0.17746858298778534	 
Topic: 0.009*"miss" + 0.009*"blaze" + 0.008*"search" + 0.007*"firefight" + 0.006*"polic" + 0.006*"damag" + 0.006*"uranium" + 0.004*"corbi" + 0.004*"restrict" + 0.004*"crew"

Score: 0.155869722366333	 
Topic: 0.017*"charg" + 0.016*"polic" + 0.012*"court" + 0.012*"murder" + 0.011*"face" + 0.007*"assault" + 0.007*"investig" + 0.007*"jail" + 0.006*"death" + 0.006*"drug"

Score: 0.012503143399953842	 
Topic: 0.008*"council" + 0.008*"plan" + 0.008*"price" + 0.007*"govt" + 0.007*"market" + 0.006*"rise" + 0.005*"telstra" + 0.005*"concern" + 0.004*"chang" + 0.004*"urg"

Score: 0.012501470744609833	 
Topic: 0.007*"final" + 0.006*"open" + 0.005*"teacher" + 0.005*"hewitt" + 0.004*"wallabi" + 0.004*"wheat" + 0.004*"cole" + 0.003*"vail" + 0.003*"corrupt" + 0.003*"lead"

Score: 0.

In [ ]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.30419841408729553	 Topic: 0.018*"home" + 0.017*"talk" + 0.017*"deal" + 0.013*"lose" + 0.012*"iraq"
Score: 0.21675674617290497	 Topic: 0.027*"plan" + 0.017*"nation" + 0.016*"urg" + 0.016*"hous" + 0.015*"public"
Score: 0.19483689963817596	 Topic: 0.023*"fear" + 0.017*"push" + 0.015*"break" + 0.014*"worker" + 0.014*"famili"
Score: 0.18420635163784027	 Topic: 0.015*"coast" + 0.015*"year" + 0.014*"gold" + 0.014*"market" + 0.013*"open"
Score: 0.01666819117963314	 Topic: 0.018*"concern" + 0.017*"govt" + 0.016*"protest" + 0.015*"school" + 0.014*"labor"
Score: 0.016666719689965248	 Topic: 0.020*"rise" + 0.019*"price" + 0.018*"say" + 0.017*"govt" + 0.017*"health"
Score: 0.01666666939854622	 Topic: 0.061*"polic" + 0.029*"kill" + 0.024*"crash" + 0.018*"death" + 0.016*"investig"
Score: 0.01666666753590107	 Topic: 0.036*"charg" + 0.034*"court" + 0.032*"face" + 0.021*"accus" + 0.020*"jail"
Score: 0.01666666753590107	 Topic: 0.034*"council" + 0.024*"water" + 0.021*"boost" + 0.017*"plan" + 0.0